In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Transformer/spam.csv")

In [4]:
df.head()

,Category,Message
0,not spam,"Go until jurong point, crazy.. Available only ..."
1,not spam,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,not spam,U dun say so early hor... U c already then say...
4,not spam,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df['Category']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)

In [6]:
df.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
X = df["Message"]
y = df["Category"]

In [8]:
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [9]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.8.*"

     |████████████████████████████████| 4.9 MB 5.1 MB/s 
     |████████████████████████████████| 462 kB 56.5 MB/s 


In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [11]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [13]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(outputs['pooled_output'])

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [14]:
model.compile(optimizer='adam',loss='binary_crossentropy')

In [15]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
122/122 [==============================] - 57s 313ms/step - loss: 0.3486
Epoch 2/5
122/122 [==============================] - 39s 322ms/step - loss: 0.2434
Epoch 3/5
122/122 [==============================] - 39s 321ms/step - loss: 0.2042
Epoch 4/5
122/122 [==============================] - 40s 325ms/step - loss: 0.1770
Epoch 5/5
122/122 [==============================] - 40s 327ms/step - loss: 0.1580


In [17]:
y_pred = model.predict(X_test)

In [20]:
y_pred = np.where(y_pred >= 0.5, 1, 0)

In [21]:
from sklearn.metrics import classification_report

In [22]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1442
           1       0.82      0.84      0.83       230

    accuracy                           0.95      1672
   macro avg       0.90      0.91      0.90      1672
weighted avg       0.95      0.95      0.95      1672

